Load Data

In [ ]:
import pandas as pd

TRAIN = pd.read_csv("train_stores.csv", parse_dates=['date'])
TEST = pd.read_csv("test_stores.csv", parse_dates=['date'])

print(TRAIN.head())

   id       date  store_nbr      family  sales  onpromotion
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0
1   1 2013-01-01          1   BABY CARE    0.0            0
2   2 2013-01-01          1      BEAUTY    0.0            0
3   3 2013-01-01          1   BEVERAGES    0.0            0
4   4 2013-01-01          1       BOOKS    0.0            0


ETS Model

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

def myf(mytest, ypred): 
    error = mytest - ypred
    MSE = np.mean(error**2)
    RMSE = MSE**0.5
    MAE = np.mean(np.abs(error))
    MAPE = np.mean(np.abs(error / (mytest + 1e-10)))  
    ME = np.mean(error)
    MPE = np.mean(error / (mytest + 1e-10))
    answer = [MSE, RMSE, MAE, MAPE, ME, MPE]
    names = ["MSE", "RMSE", "MAE", "MAPE", "ME", "MPE"]
    return dict(zip(names, answer))


TRAIN = TRAIN.sort_values(['store_nbr', 'family', 'date'])

model_results = {}
metrics_by_group = {}

for (store, family), group in TRAIN.groupby(['store_nbr', 'family']):
    try:
        
        ts = group.set_index('date').resample('D')['sales'].sum().fillna(0)

        
        if ts.sum() == 0 or ts.nunique() == 1 or len(ts) < 30:
            print(f"Skipping store {store}, family {family} — insufficient data")
            continue

       
        model = ETSModel(ts, error="add", trend="add", seasonal="add",
                         damped_trend=True, seasonal_periods=7)
        fit = model.fit()

        
        model_results[(store, family)] = {
            'model': fit,
            'actual': ts,
            'predicted': fit.fittedvalues
        }

        
        actual = ts
        predicted = fit.fittedvalues

        common_index = actual.index.intersection(predicted.index)
        mytest = actual.loc[common_index]
        ypred = predicted.loc[common_index]

        metrics = myf(mytest, ypred)
        metrics_by_group[(store, family)] = metrics

    except Exception as e:
        print(f"Failed for store {store}, family {family}: {e}")


metrics_df = pd.DataFrame(metrics_by_group).T.reset_index()
metrics_df.columns = ['store_nbr', 'family', 'MSE', 'RMSE', 'MAE', 'MAPE', 'ME', 'MPE']


Skipping store 1, family BABY CARE — insufficient data
Skipping store 9, family BOOKS — insufficient data
Skipping store 10, family BOOKS — insufficient data
Skipping store 11, family BOOKS — insufficient data
Skipping store 12, family BOOKS — insufficient data
Skipping store 13, family BABY CARE — insufficient data
Skipping store 13, family BOOKS — insufficient data
Skipping store 14, family BOOKS — insufficient data
Skipping store 14, family LAWN AND GARDEN — insufficient data
Skipping store 15, family BOOKS — insufficient data
Skipping store 16, family BOOKS — insufficient data
Skipping store 16, family LADIESWEAR — insufficient data
Skipping store 17, family BOOKS — insufficient data
Skipping store 18, family BOOKS — insufficient data
Skipping store 19, family BOOKS — insufficient data
Skipping store 20, family BOOKS — insufficient data
Skipping store 21, family BOOKS — insufficient data
Skipping store 22, family BOOKS — insufficient data
Skipping store 23, family BABY CARE — insuf

Evaluation of Average Metrics

In [3]:
average_metrics = metrics_df[['MSE', 'RMSE', 'MAE', 'MAPE', 'ME', 'MPE']].mean()
print(average_metrics)


MSE     8.810221e+04
RMSE    1.087056e+02
MAE     5.952917e+01
MAPE    3.160015e+10
ME      7.424398e-01
MPE    -2.708965e+10
dtype: float64


Fitting ETS Model on Test Data

In [ ]:


TEST = TEST.sort_values(['store_nbr', 'family', 'date'])


forecast_rows = []


for (store, family), group in TEST.groupby(['store_nbr', 'family']):
    key = (store, family)

    temp = group[['id']].copy()

    if key in model_results:
        try:
            
            fitted_model = model_results[key]['model']

            
            n_periods = len(group)
            forecast = fitted_model.forecast(n_periods)

            temp['sales'] = forecast.values

        except Exception as e:
            print(f"Forecast failed for store {store}, family {family}: {e}")
            
            temp['sales'] = 0

    else:
        print(f"No model found for store {store}, family {family}")
        
        temp['sales'] = 0

    forecast_rows.append(temp)


submission_df = pd.concat(forecast_rows)




No model found for store 1, family BABY CARE
No model found for store 9, family BOOKS
No model found for store 10, family BOOKS
No model found for store 11, family BOOKS
No model found for store 12, family BOOKS
No model found for store 13, family BABY CARE
No model found for store 13, family BOOKS
No model found for store 14, family BOOKS
No model found for store 14, family LAWN AND GARDEN
No model found for store 15, family BOOKS
No model found for store 16, family BOOKS
No model found for store 16, family LADIESWEAR
No model found for store 17, family BOOKS
No model found for store 18, family BOOKS
No model found for store 19, family BOOKS
No model found for store 20, family BOOKS
No model found for store 21, family BOOKS
No model found for store 22, family BOOKS
No model found for store 23, family BABY CARE
No model found for store 25, family LADIESWEAR
No model found for store 28, family BOOKS
No model found for store 28, family LADIESWEAR
No model found for store 29, family BOOKS

In [ ]:
print(submission_df.head())

#submission_df.to_csv("ets_submission.csv", index=False)

ETS Assumptions:

The time series is composed of predictable components: error, trend, and seasonality FAILED.

1. residual mean: if not close to or equal to 0 this indicates a pattern exists suggesting that the model failed to fully capture the trend 

2. Lb_pvalue: Ljung-Box test pvalue where less than 0.05 indicates autocorrelation or a pattern in the residuals, i.e ETS missed some predictable trends.

3. If one or two fail in the assumption dataframe this is considered an overall failure by our model. The fails and successes are captured by column overall_pass. The pass rate of 14.63% indicates only about 15% of correctly capture predictable components.


In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox


assumption_results = []

for key, result in model_results.items():
    store, family = key
    actual = result['actual']
    predicted = result['predicted']
    residuals = actual - predicted

    
    residual_mean = residuals.mean()

   
    lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)
    lb_pvalue = lb_test['lb_pvalue'].iloc[0]

   
    mean_pass = np.abs(residual_mean) < 0.5  # Residual mean close to 0
    lb_pass = lb_pvalue > 0.05                # No autocorrelation

    overall_pass = mean_pass and lb_pass

    
    assumption_results.append({
        'store_nbr': store,
        'family': family,
        'residual_mean': residual_mean,
        'lb_pvalue': lb_pvalue,
        'mean_pass': mean_pass,
        'autocorr_pass': lb_pass,
        'overall_pass': overall_pass
    })


assumption_df = pd.DataFrame(assumption_results)


print(f"\nOverall ETS Assumption Pass Rate: {assumption_df['overall_pass'].mean() * 100:.2f}%")
assumption_df.head()



Overall ETS Assumption Pass Rate: 14.63%


,store_nbr,family,residual_mean,lb_pvalue,mean_pass,autocorr_pass,overall_pass
0,1,AUTOMOTIVE,0.074484,4.500514e-01,True,True,True
1,1,BEAUTY,0.025466,6.403088e-02,True,True,True
2,1,BEVERAGES,1.803869,3.243557e-14,False,False,False
3,1,BOOKS,0.000298,2.663330e-21,True,False,False
4,1,BREAD/BAKERY,0.073867,4.668787e-09,True,False,False


ARIMA Model

In [ ]:

TRAIN = pd.read_csv("train_stores.csv", parse_dates=['date'])
TEST = pd.read_csv("test_stores.csv", parse_dates=['date'])

print(TRAIN.head())

   id       date  store_nbr      family  sales  onpromotion
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0
1   1 2013-01-01          1   BABY CARE    0.0            0
2   2 2013-01-01          1      BEAUTY    0.0            0
3   3 2013-01-01          1   BEVERAGES    0.0            0
4   4 2013-01-01          1       BOOKS    0.0            0


In [ ]:
import pandas as pd
import numpy as np
import warnings
from statsmodels.tsa.arima.model import ARIMA

warnings.filterwarnings("ignore")


def myf(mytest, ypred): 
    error = mytest - ypred
    MSE = np.mean(error**2)
    RMSE = MSE**0.5
    MAE = np.mean(np.abs(error))
    MAPE = np.mean(np.abs(error / (mytest + 1e-10)))
    ME = np.mean(error)
    MPE = np.mean(error / (mytest + 1e-10))
    answer = [MSE, RMSE, MAE, MAPE, ME, MPE]
    names = ["MSE", "RMSE", "MAE", "MAPE", "ME", "MPE"]
    return dict(zip(names, answer))


TRAIN = TRAIN.sort_values(['store_nbr', 'family', 'date'])


arima_results = {}
metrics_by_group = {}

for (store, family), group in TRAIN.groupby(['store_nbr', 'family']):
    try:
        ts = group.set_index('date').resample('D')['sales'].sum().fillna(0)

        
        if ts.nunique() <= 1 or len(ts) < 30:
            continue

        
        model = ARIMA(ts, order=(1, 1, 1))
        fit = model.fit()

       
        fitted = fit.fittedvalues

        
        common_index = ts.index.intersection(fitted.index)
        mytest = ts.loc[common_index]
        ypred = fitted.loc[common_index]

        
        arima_results[(store, family)] = {
            'model': fit,
            'actual': ts,
            'predicted': fitted
        }

        metrics = myf(mytest, ypred)
        metrics_by_group[(store, family)] = metrics

    except Exception as e:
        print(f"Failed for store {store}, family {family}: {e}")


In [17]:
metrics_df = pd.DataFrame(metrics_by_group).T.reset_index()
metrics_df.columns = ['store_nbr', 'family', 'MSE', 'RMSE', 'MAE', 'MAPE', 'ME', 'MPE']

average_metrics = metrics_df[['MSE', 'RMSE', 'MAE', 'MAPE', 'ME', 'MPE']].mean()
print(average_metrics.round(4))

MSE     1.403435e+05
RMSE    1.380879e+02
MAE     8.913570e+01
MAPE    3.012617e+10
ME      3.542400e+00
MPE    -3.012606e+10
dtype: float64


In [ ]:

TEST = TEST.sort_values(['store_nbr', 'family', 'date'])


forecast_rows = []


for (store, family), group in TEST.groupby(['store_nbr', 'family']):
    key = (store, family)
    temp = group[['id']].copy()

    if key in arima_results:
        try:
            model = arima_results[key]['model']
            n_periods = len(group)
            forecast = model.forecast(steps=n_periods)

            temp['sales'] = forecast.values

        except Exception as e:
            print(f"Forecast failed for store {store}, family {family}: {e}")
            temp['sales'] = 0
    else:
        print(f"No ARIMA model found for store {store}, family {family}")
        temp['sales'] = 0

    forecast_rows.append(temp)


submission_df = pd.concat(forecast_rows)




No ARIMA model found for store 1, family BABY CARE
No ARIMA model found for store 9, family BOOKS
No ARIMA model found for store 10, family BOOKS
No ARIMA model found for store 11, family BOOKS
No ARIMA model found for store 12, family BOOKS
No ARIMA model found for store 13, family BABY CARE
No ARIMA model found for store 13, family BOOKS
No ARIMA model found for store 14, family BOOKS
No ARIMA model found for store 14, family LAWN AND GARDEN
No ARIMA model found for store 15, family BOOKS
No ARIMA model found for store 16, family BOOKS
No ARIMA model found for store 16, family LADIESWEAR
No ARIMA model found for store 17, family BOOKS
No ARIMA model found for store 18, family BOOKS
No ARIMA model found for store 19, family BOOKS
No ARIMA model found for store 20, family BOOKS
No ARIMA model found for store 21, family BOOKS
No ARIMA model found for store 22, family BOOKS
No ARIMA model found for store 23, family BABY CARE
No ARIMA model found for store 25, family LADIESWEAR
No ARIMA m

In [ ]:
submission_df.head()

#submission_df.to_csv("arima_submission.csv", index=False)

,id,sales
0,3000888,4.452096
1782,3002670,4.453301
3564,3004452,4.453304
5346,3006234,4.453304
7128,3008016,4.453304


ARIMA assumptions:

Stationary mean, variance, and no autocorrelation

ADF Test: Test for stationary mean. If p-val less than 0.05 its stationary

Ljung-Box Test: Test for autocorrelation. None if p-value > 0.05

Breusch-Pagan Test: Test for constant variance. Constant variance if p-val>0.05

If all three pass then that model passes assumptions. Only 4.8% pass assumption.

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox, het_breuschpagan
import statsmodels.api as sm


assumption_results = []

for key, result in arima_results.items():
    store, family = key
    actual = result['actual']
    predicted = result['predicted']
    residuals = actual - predicted

   
    residuals = residuals.dropna()

    
    adf_stat, adf_pvalue, _, _, _, _ = adfuller(residuals)

    
    lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)
    lb_pvalue = lb_test['lb_pvalue'].iloc[0]

    
    X = sm.add_constant(np.arange(len(residuals)))  
    bp_test = het_breuschpagan(residuals, X)
    bp_pvalue = bp_test[1]

    
    adf_pass = adf_pvalue < 0.05           # Stationary mean
    lb_pass = lb_pvalue > 0.05              # No autocorrelation 
    bp_pass = bp_pvalue > 0.05              # Constant variance

    overall_pass = adf_pass and lb_pass and bp_pass

    
    assumption_results.append({
        'store_nbr': store,
        'family': family,
        'adf_pvalue': adf_pvalue,
        'lb_pvalue': lb_pvalue,
        'bp_pvalue': bp_pvalue,
        'stationary_mean_pass': adf_pass,
        'no_autocorr_pass': lb_pass,
        'constant_variance_pass': bp_pass,
        'overall_pass': overall_pass
    })


assumption_df = pd.DataFrame(assumption_results)


print(f"\nOverall ARIMA Assumption Pass Rate: {assumption_df['overall_pass'].mean() * 100:.2f}%")
assumption_df.head()



Overall ARIMA Assumption Pass Rate: 4.80%


,store_nbr,family,adf_pvalue,lb_pvalue,bp_pvalue,stationary_mean_pass,no_autocorr_pass,constant_variance_pass,overall_pass
0,1,AUTOMOTIVE,8.438795e-14,8.384527e-03,1.031470e-04,True,False,False,False
1,1,BEAUTY,6.030752e-19,3.577693e-03,2.705766e-10,True,False,False,False
2,1,BEVERAGES,1.209068e-15,3.587820e-135,2.658162e-15,True,False,False,False
3,1,BOOKS,4.769227e-11,3.189858e-17,1.117712e-13,True,False,False,False
4,1,BREAD/BAKERY,1.823979e-11,3.287614e-193,3.230972e-07,True,False,False,False
